In [13]:
import pandas as pd
import numpy as np
from typing import Dict

In [14]:
def window_append(window, nbhd, x):
    if len(window) > nbhd:
        window.pop(0)
    window.append(x)

def window_append_list(x: list):
    for i in x:
        window_append(i)

def calcResidual(pred, actual) -> float:
    return actual - pred

# takes a list with (usually) timestamp and price columns
# returns [a, b] from the regression equation y = ax + b
def regression(z) -> tuple[float, float]:
    n = len(z)
    x = []
    for i in range(n):
        x.append(i)
    y = np.array(z)
    a, b = np.polyfit(x, y, 1)
    return a, b

# takes a list and returns a smoothed list of the same size
def filter(smoothing, z: list) -> list:
    z_doubled = z

    z_fft = np.fft.fft(z_doubled)
    n = len(z_doubled)
    filter = []
    for i in range(n):
        if i < (n / 2) *  (1 - smoothing) or i > (n / 2) * (1 + smoothing):
            filter.append(1)
        else:
            filter.append(0)
    z_smoothed = np.fft.ifft(z_fft * filter)
    z1 = list(z_smoothed.flatten())
    return z1


In [15]:
imcDF = pd.read_csv('../IMC_data.csv')
imcDF.set_index('timestamp', inplace=True)
starfruitDF = imcDF[imcDF['product'] == 'STARFRUIT']

In [16]:
bestReturns = -float('inf')
bestSmoothing = None
bestK = None
bestTimestampDelta = None
bestNbhd = None

for smoothing in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
    for K in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]:
        for timestampDelta in range(100, 10000, 100):
            for nbhd in range (2, 64, 2):

                tradeLog = []
                positionOpen = False
                positionType = None
                window = []

                endRow = 1
                for timestamp, row in starfruitDF.iterrows():

                    curData = starfruitDF.iloc[:endRow]

                    curAsk = row['ask_price_1']
                    curAskVolume = row['ask_volume_1']
                    curBid = row['bid_price_1']
                    curBidVolume = row['bid_volume_1']

                    window_append(window, nbhd, curAsk)
                    if len(window) > nbhd:
                        winSmoothed = filter(smoothing, window)
                        a, b = regression(winSmoothed)
                        residual = calcResidual(curAsk, a * len(window) + b)
                    else:
                        residual = 0
                        
                    if (timestamp / timestampDelta) > nbhd and residual < K and not positionOpen:
                         
                        positionOpen = True
                        positionType = 'long'
                        tradeLog.append({
                            'timestamp': timestamp,
                            'action': 'open position',
                            'price': curAsk,
                            'type': 'long',
                            'shares': 1
                        })
                    
                    elif residual > -K and positionOpen:
                        positionOpen = False
                        positionType = None
                        tradeLog.append({
                            'timestamp': timestamp,
                            'action': 'close position',
                            'price': curBid,
                            'type': 'long',
                            'shares': 1
                        })
   
                    endRow += 1

                tradeLogDF = pd.DataFrame(tradeLog)
                totalReturn = 0
                open_position = None
                close_position_dates = []
                close_position_returns = []

                for index, trade in tradeLogDF.iterrows():
                    if trade['action'] == 'open position':
                        open_position = trade
                    if trade['action'] == 'close position':

                        close_position_dates.append(trade['timestamp'])
                        profitLoss = (trade['price'] - open_position['price']) * open_position['shares']
                        close_position_returns.append(profitLoss)
                        totalReturn += profitLoss
                        open_position = None
                    
                    if totalReturn > bestReturns:
                        bestReturns = totalReturn
                        bestSmoothing = smoothing
                        bestK = K
                        bestTimestampDelta = timestampDelta
                        bestNbhd = nbhd

print(f'Best returns: {bestReturns}')
print(f'Best smoothing: {bestSmoothing}')
print(f'Best K: {bestK}')
print(f'Best timestamp delta: {bestTimestampDelta}')
print(f'Best neighbourhood: {bestNbhd}')
